# Wrangling

In [3]:
library(dplyr)

In [7]:
vancouver_crime_data <- read.csv("crimedata_csv_AllNeighbourhoods_AllYears.csv")
head(vancouver_crime_data)
cat("Number of Crimes: ", nrow(vancouver_crime_data))

,TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
,<chr>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>
1,Other Theft,2019,3,14,10,34,47XX OAK ST,South Cambie,490706.9,5454474
2,Other Theft,2005,8,1,0,0,47XX OAK ST,South Cambie,490706.9,5454474
3,Other Theft,2012,12,6,9,30,47XX OAK ST,South Cambie,490706.9,5454474
4,Other Theft,2004,1,25,17,0,47XX OAK ST,South Cambie,490710.6,5454535
5,Other Theft,2020,9,22,11,36,47XX OSLER ST,Shaughnessy,490528.5,5454450
6,Other Theft,2003,8,18,7,30,47XX OSLER ST,Shaughnessy,490529.8,5454502


Number of Crimes:  853296

In [11]:
vancouver_thefts <- vancouver_crime_data %>%
    filter(TYPE == "Other Theft" | TYPE == "Theft from Vehicle" | TYPE == "Theft of Vehicle" | TYPE == "Theft of Bicycle") %>%
    mutate(DAY_NIGHT = ifelse(HOUR >= 6 & HOUR < 18, "DAY", "NIGHT")) %>%
    select(TYPE, YEAR, HOUR, DAY_NIGHT, NEIGHBOURHOOD)
head(vancouver_thefts)

theft_counts <- vancouver_thefts %>%
    group_by(TYPE) %>%
    summarise(COUNT = n())
theft_counts

,TYPE,YEAR,HOUR,DAY_NIGHT,NEIGHBOURHOOD
,<chr>,<int>,<int>,<chr>,<chr>
1,Other Theft,2019,10,DAY,South Cambie
2,Other Theft,2005,0,NIGHT,South Cambie
3,Other Theft,2012,9,DAY,South Cambie
4,Other Theft,2004,17,DAY,South Cambie
5,Other Theft,2020,11,DAY,Shaughnessy
6,Other Theft,2003,7,DAY,Shaughnessy


TYPE,COUNT
<chr>,<int>
Other Theft,210540
Theft from Vehicle,237891
Theft of Bicycle,36403
Theft of Vehicle,44404
